In [ ]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.auth import TokenAuthentication
from codeflare_sdk.job.jobs import DDPJobDefinition

First, we begin by authenticating using the SDK.

In [ ]:
# Create authentication object for oc user permissions
auth = TokenAuthentication(
    token = "sha256~VbcZdJTD-ZtgIry4HE_JobjQGRM6bXkn45q-gIGdajw",
    server = "https://api.mini2.mydomain.com:6443",
    skip_tls=True
)
auth.login()

Now that we are logged in, we can directly submit our batch job (model training on two workers with four gpus each) to MCAD via torchx.

In [ ]:
import os
jobdef = DDPJobDefinition(
    name="mnistjob",
    script="mnist_fashion.py",
    scheduler_args={"namespace": "huggingface"},
    j="2x1",
    gpu=1,
    cpu=1,
    memMB=8000,
    env={'AWS_ACCESS_KEY_ID':os.environ.get('AWS_ACCESS_KEY_ID'),
         'AWS_SECRET_ACCESS_KEY':os.environ.get('AWS_SECRET_ACCESS_KEY'),
         'AWS_S3_ENDPOINT':os.environ.get('AWS_S3_ENDPOINT'),
         'OUTPUT_PATH':'mymodel.onnx'},
    image="image-registry.openshift-image-registry.svc:5000/huggingface/test2:latest"
    #image="quay.io/michaelclifford/mnist-test:latest"
)
job = jobdef.submit()

Now we can go ahead and look at the status and logs of our batch job.

In [ ]:
job.status()

In [ ]:
print(job.logs())

Finally, we can remove the job once complete and release/terminate the associated resources, bringing everything back to the way it was before job submission.

In [ ]:
job.cancel()

In [ ]:
auth.logout()